In [1]:
import pandas as pd
from r import *

data = pd.read_csv("./dataset.csv")
formule = data['Equation']

for f in range(2, 10):
    tokens = parse_equation1(formule[f])
    fix_sorted_tokens = fix2(tokens)
    sorted_tokens = sort_tokens(fix_sorted_tokens)

    tree = build_tree(sorted_tokens)
    rez = calculate_tree(tree, 2)
    print(formule[f],rez )
    # tree = build_tree(sorted_tokens)
    # print_infix(tree)


['x', '(', '3', '4', '-', ')', '+']
x + 3 - 4 1
(((x - 1) - 3) / 5) ['(', '(', '(', '(', 'x', '1', '-', ')', '3', '-', ')', '5', '/', ')', ')']
(x * 5) ['(', '(', 'x', '5', '*', ')', ')']
(x + 2) ['(', 'x', '2', '+', ')']
((x + 1) * 7) ['(', '(', '(', 'x', '1', '+', ')', '7', '*', ')', ')']
((x + 2) ** 9) ['(', '(', 'x', '2', '+', ')', '9', '^', ')']
(x * 2) ['(', '(', 'x', '2', '*', ')', ')']
(((x ** 8) * 5) / 3) ['(', '(', '(', '(', '(', 'x', '8', '^', ')', '5', '*', ')', ')', '3', '/', ')', ')']
(x / 7) ['(', '(', 'x', '7', '/', ')', ')']
